Register on [modal.com](https://modal.com/signup) and avail of $30 free credit per month.

Then run:
```bash
pip install modal
modal setup  # to authenticate (if this doesn’t work, try python -m modal setup)
```

In [1]:
from nemotron_inference_modal_9B import serve, app
import urllib

app.app_id

Let's see if we can find our app for serving the Nemotron Nano 9B model. If not, let's deploy it.

In [2]:
if not app.app_id:
    try:
        app.lookup("nemotron-nano-9B-v2-inference")
    except:
        app.deploy()

app.app_id

'ap-z3NEtW68yuO8zUB5Fz1po8'

Because our `serve` `Function` (defined in `nemotron_inference_modal_9B.py`) has the `@modal.web_server(port=VLLM_PORT, startup_timeout=10 * MINUTES)` decorator, it comes equipped with a web server we can spin up by hitting its `url`.

Let's first get the `url` for the `serve` `Function`.

In [3]:
import modal
f = modal.Function.from_name("nemotron-nano-9B-v2-inference", "serve")
url = f.get_web_url()
url

'https://rosmulski--nemotron-nano-9b-v2-inference-serve.modal.run'

We can hit the `/health` endpoint to bring the app up.

The first time around, tt can take ~3 minutes.

Once that's done, our app will stay up as long as it receives a request within `scaledown_window` seconds (5 minutes in our case, given tha params passed to the `@app.function` decorator).

In [4]:
with urllib.request.urlopen(f"{url}/health") as response:
    data = response.read().decode('utf-8')
    if response.status == 200:
        print("We are up and running!")
    else:
        print(f"Health check failed. Review logs at modal.com/apps.\nResponse: {data}")

We are up and running!


And we are ready to talk to our LLM! :)

Let's check the available models (should be just our `NVIDIA-Nemotron-Nano-9B-v2`).

In [5]:
from openai import OpenAI
client = OpenAI(base_url=url + '/v1')
client.models.list()

SyncPage[Model](data=[Model(id='nvidia/NVIDIA-Nemotron-Nano-9B-v2', created=1757930137, object='model', owned_by='vllm', root='nvidia/NVIDIA-Nemotron-Nano-9B-v2', parent=None, max_model_len=131072, permission=[{'id': 'modelperm-2b0c65b86e37479e9405cb4020cb0a02', 'object': 'model_permission', 'created': 1757930137, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')

In [41]:
response = client.responses.create(
    model="nvidia/NVIDIA-Nemotron-Nano-9B-v2",
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

print(response.output_text)

Okay, the user is asking if semicolons are optional in JavaScript. Let me start by recalling what I know about JavaScript syntax.

Semicolons are used to terminate statements, but I remember that JavaScript has automatic semicolon insertion (ASI). So the engine might add a semicolon where it's needed. But wait, does that mean they're optional?

Hmm, but there are some cases where you have to use them. Like when you're writing multiple statements on the same line. For example, if you have 'var a = 1; var b = 2;' versus 'var a = 1 var b = 2;'—the latter would be parsed as 'var a = 1' and 'var b = 2' because of ASI. But if you write them without semicolons on the same line, it might cause issues. So in that case, are semicolons required?

Also, in some code structures, like when using commas or other characters that could be ambiguous, semicolons might be necessary to prevent ASI from adding them in the wrong place. For instance, if you have 'function foo() { return 1 + 2' without a semic

Two things become apparent:
1. Nemotron 9B cannot talk like a pirate!
    - This in fact might be a good thing. There has been a lot of talk recently how "reasoning models" lack personality and have been hyper-tuned to squeeze out as much intelligence on certain tasks (math, programming, etc) from the weights as possible. This is exactly what I am hoping to get from this model. In that light, this behavior is a good sign.
2. You can't use API calls like this in your code.

Let me elaborate on the 2nd point.

How do we ensure we use more of something? We reduce the friction around using said thing.

We could wrap the `client.responses.create` calls into functions (and build the parsing tooling around it). But that strikes me as a lot of work and still I wouldn't be able to come up on my own with something even remotely as awesome as the library I am going to introduce you to next!

I nearly universally hate AI frameworks and SDKs, but I have fallen in love with `BAML`.

The problem I was trying to solve was: "how do I call any model in a simple yet powerful way?" What if I would like to get structured output and tool calling?

Some of the libraries that I tried came close, but none provided the entire functionality. `BAML` doesn't only meet the above criteria, but it also brings [something else that is very unique and valuable](https://boundaryml.com/blog/sota-function-calling?q=0) in its own right.

Let me show you what you can do with it.

# Basic Calls

To be able to execute the cells below, please follow the installation steps outlined [here](https://docs.boundaryml.com/guide/installation-language/python)

The following cell performs some config that is generally not necessary (you can specify all this information via `baml_src/clients.baml`).

But by adding this cell, I am saving you the need to open the `clients.baml` file and edit it just for the purposes of this demo.

In [7]:
from baml_py import ClientRegistry
from baml_py.baml_py import set_log_level
set_log_level("ERROR")

cr = ClientRegistry()
cr.add_llm_client(name='NemotronOnModal', provider='openai-generic', options={
        "model": "nvidia/NVIDIA-Nemotron-Nano-9B-v2",
        "base_url": url + '/v1',
        "api_key": ""
    })

cr.set_primary('NemotronOnModal')

[BAML] Log level set to ERROR


## Standard Completion

The output of the model is quite verbose  we are not — we are not applying any parsing to it. This is just for demonstration, we would never use the model like this in actual code.

In [13]:
from baml_client import b
from IPython.display import Markdown

r = b.GetCompletion("Are semicolons optional in JavaScript?", { "client_registry": cr })
Markdown(r)

Okay, the user is asking if semicolons are optional in JavaScript. Let me start by recalling what I know about semicolons in JS.

First, I remember that JavaScript is a loosely typed language, and its syntax allows for certain flexibility. Semicolons are used to terminate statements, right? Like in many languages, you put them at the end of a line to indicate the end of a command.

But wait, JavaScript has something called Automatic Semicolon Insertion (ASI). From what I've studied, this feature allows the interpreter to add a semicolon where it's missing if it can infer the end of a statement. So in cases where you forget to put a semicolon, the engine might automatically add one. However, this isn't always reliable. There are situations where ASI can fail, leading to unexpected behavior.

For example, if a line ends with a certain character that could be ambiguous, ASI might not add the semicolon correctly. Like if a line ends with a comma or a closing parenthesis, the parser might not know where the statement ends. That could cause errors later on.

So even though semicolons are technically optional because of ASI, relying on them is discouraged. It's considered a bad practice because it introduces ambiguity. Developers are advised to use semicolons explicitly to avoid potential bugs, especially when writing code that might be minified or processed by tools that could alter the code in ways that remove or add semicolons.

Another point to consider is consistency. If a developer sometimes uses semicolons and sometimes doesn't, it can make the code harder to read or maintain. Also, different environments or code minifiers might behave differently with ASI, leading to inconsistent results.

Are there specific cases where semicolons are required even with ASI? Yes. For instance, if you have two statements on the same line without a semicolon, it might concatenate them into one statement if ASI can't figure out where to split them. Like var a = 1; var b = 2; versus var a = 1 var b = 2; the latter might become var a = 1var b = 2; which is a syntax error.

Also, when using certain operators or expressions that end with a symbol that looks like a statement, ASI might not work as intended. For example, ending a line with a function parameter list or an object literal.

So putting it all together, the answer is that yes, semicolons are optional in JavaScript due to ASI, but it's not recommended to rely on that. Best practice is to use them explicitly to prevent potential issues.

Wait, but I should check if there are any edge cases or recent changes. ES6 or later versions didn't change the ASI rules, I think. So the core idea holds. Also, maybe mention that in some cases, even with ASI, you need to add semicolons, like after a return statement or in certain function definitions.

Another example: if you have a line that's a single expression and then another line that starts with a variable declaration, without a semicolon, it could be parsed incorrectly. Like function foo() { var x = 5; var y = 10; } versus function foo() { var x =5 var y=10; }, which would merge the var statements into one, but ASI might not catch that. Wait, in that case, ASI would add a semicolon after the 5 and then see the var y... as a new statement. Wait, maybe not. If you have "var x=5 var y=10", ASI would add a semicolon after the 5, making it "var x=5; var y=10", which is correct. Hmm. Maybe that example isn't the best. Another example could be when a line ends with a comma, which might not trigger ASI properly.

Alternatively, if you have something like:

var obj = { a: 1
b: 2 };

Here, without a semicolon after the first line, ASI might not add it correctly, leading to a syntax error.

In conclusion, while semicolons are optional in JavaScript due to ASI, relying on them can lead to bugs, so it's better to include them explicitly.
</think>

Yes, semicolons are **technically optional** in JavaScript due to **Automatic Semicolon Insertion (ASI)**, a feature of the JavaScript engine that adds missing semicolons in some cases. However, relying on ASI is **not recommended** due to potential ambiguity and bugs. Here's a breakdown:

### Key Points:
1. **Automatic Semicolon Insertion (ASI):**
   - The JavaScript engine automatically adds semicolons where possible to terminate statements.
   - Example: `var x = 5;` can be written as `var x = 5`.

2. **When ASI Works:**
   - At the end of a line (common case).
   - When the line ends with a `}`, `)`, or `]` (e.g., closing brackets).

3. **When ASI Fails:**
   - When the line ends with a character or structure that could be misinterpreted (e.g., commas, certain operators).
   - Example: Forgetting a semicolon between statements on the same line:
     ```javascript
     var a = 1 var b = 2; // ❌ Becomes `var a = 1var b = 2;` (invalid)
     ```
   - When lines end with commas or other ambiguous characters.

4. **Best Practices:**
   - **Always use semicolons explicitly** to avoid reliance on ASI.
   - Enforce consistent coding style (e.g., ESLint rules can enforce semicolons).
   - Semicolons prevent parsing errors, especially in complex or minified code.

### Why Avoid Relying on ASI?
- **Ambiguity:** The engine might misinterpret where a statement ends.
- **Minifiers:** Tools may remove semicolons during compression, breaking code.
- **Readability:** Explicit semicolons improve code clarity for others (and future you).

### Conclusion:
While semicolons are optional in theory, they are essential in practice for robust, maintainable, and error-free JavaScript code.


## Structured Output

Very often you want to get something specific from a model:
* a yes or no answer
* a score
* a piece of code that achieves something

`BAML` makes this super convenient plus it adds a bit of secret sauce so that the experience and performance you get [is better](https://boundaryml.com/blog/structured-output-from-llms) to what you might expect when using standard SDKs.

Let's look at an example.

Below we have text taken from [HF's model card](https://huggingface.co/nvidia/NVIDIA-Nemotron-Nano-9B-v2) for the Nemotron model we are using:

In [14]:
text = """\
NVIDIA-Nemotron-Nano-9B-v2 is a large language model (LLM) trained from scratch by NVIDIA, and designed as a unified model for both reasoning and non-reasoning tasks. It responds to user queries and tasks by first generating a reasoning trace and then concluding with a final response. The model's reasoning capabilities can be controlled via a system prompt. If the user prefers the model to provide its final answer without intermediate reasoning traces, it can be configured to do so, albeit with a slight decrease in accuracy for harder prompts that require reasoning. Conversely, allowing the model to generate reasoning traces first generally results in higher-quality final solutions to queries and tasks.

The model uses a hybrid architecture consisting primarily of Mamba-2 and MLP layers combined with just four Attention layers. For the architecture, please refer to the Nemotron-H tech report. The model was trained using Megatron-LM and NeMo-RL.

The supported languages include: English, German, Spanish, French, Italian, and Japanese. Improved using Qwen.

This model is ready for commercial use.
"""

Let's see if we can extract:
- the libraries the model was trained 
- the supported langauges

In [16]:
b.ExtractLibrariesAndLanguages(text, { "client_registry": cr })

LibrariesAndLanguages(used_libraries=['Megatron-LM', 'NeMo-RL'], supported_languages=['English', 'German', 'Spanish', 'French', 'Italian', 'Japanese'])

At this point, you might be wondering where do the functions like the `ExtractLibrariesAndLanguages` above come from? You define them using `BAML` — a domain specific langauge — in `baml_src`.

But doesn't having to mess with files on disk make my life harder? Why can't I define everything inline?

`BAML` comes with a lot of streamlined tools, like the ability to define test cases anditerate on your functionality directly in VS Code, helpful syntax parsers, etc.

There is not an ounce of functionality that feels out of place or that was put in with the intention of achieving anything else than programmer productivity and possibly even happiness. I suspect 3 things:
* `BoundaryML` are not your run-of-the-mill programmers you are likely to run into at the "JAVA Programming Patterns Weekly Appreciation Club" 
* they are probably developing `BAML` for internal needs (dogfooding does wonders to your software!)
* they really know what they are doing (as evidenced on their superb [YT channel](https://www.youtube.com/@boundaryml))

Let's finish strong by building a small chat bot and showcasing tool calling (both taken from `BAML` [examples](https://docs.boundaryml.com/examples/interactive-examples))

## Chat Interface

In [17]:
from baml_client import b
from baml_client.types import MyUserMessage

messages: list[MyUserMessage] = []

while True:
    content = input("Enter your message (or 'quit' to exit): ")
    if content.lower() == 'quit':
        break
    print(f"You: {content}")
    messages.append(MyUserMessage(role="user", content=content))
    
    agent_response = b.ChatWithLLM(messages=messages);
    print(f"AI: {agent_response}")
    print()
    
    # Add the agent's response to the chat history
    messages.append(MyUserMessage(role="assistant", content=agent_response))

You: Hi, my name is Radek.
AI: Hi Radek! Nice to meet you. How can I help you today? If you tell me what you’re working on or what you’re curious about, I’ll tailor my assistance.

You: What is the meaning of life?
AI: Hi Radek. Great question, and there isn’t one universal answer. Here are a few common perspectives:

- Personal meaning: Many people find meaning by deepening relationships, growing as a person, and helping others.
- Existential view: Life may not have inherent meaning, but we get to create our own purpose through our choices and how we live.
- Philosophical angles: Some find meaning in pursuing virtues or honing their talents (eudaimonia); others embrace the idea of the absurd and still choose to live fully.
- Spiritual/religious view: Meaning is often seen as aligning with a larger purpose, God, or a moral framework.
- Scientific view: The universe isn’t designed with a meaning, but we can derive meaning from curiosity, wonder, and the impact we have on others.

A prac

## Tool Calling

Tool calling in `BAML` is... unusual.

### What I like about it

You should always think at least twice whether you really need an agent. Most of the time, structured flow is the way to go (where the orchestration of what happens is done in code, not handled via the LLM).

`BAML` gives you the tools to fill in the gap between no tool calling at all and fully agentic tool calling. Meaning, you can give your model as much *agency* as you believe is most conductive to achieving your goals.

### What I am concerned about

As long-horizon agentic workflows are going to be trained more and more into the models, will this way of tool calling continue to be the best? But I have very little experience with this approach — maybe that is not something worth worrying at this point in time.

Without further ado, let's give tool calling using the `BAML` way a go!

In [18]:
b.ToolCallOrMessage('How are you doing today?', { "client_registry": cr })

Message(content="I'm just a language model, so I don't have feelings or a physical state. But I'm here and ready to help you with any questions you might have!")

In [19]:
r = b.ToolCallOrMessage('What is the weather like in Brisbane?', { "client_registry": cr })
r

GetWeatherAPI(location='Brisbane')

In [40]:
if isinstance(r, str):
    print("AI:", r)
else:
    print(f"Tool Call... Call {r.__class__.__name__} passing in {r} and optionally use this information in subsequent calls to the model")

Tool Call... Call GetWeatherAPI passing in location='Brisbane' and optionally use this information in subsequent calls to the model


On one hand, we get greater flexiblity with this approach. The pattern where:
```
prompt -> tool call requested by the model -> tool result is passed back to the model -> model returns a response or another tool call
```
is just one of many that we can implement.

On the other hand, having a tool loop where the schema is inferred from a Python function (or multiple functions) passed to the model, would probably be nice.

Again, it all goes back to the notion that very likely, for most use cases today, one would still be better off creating a workflow rather than building an agent.

This is probably still true, even if we could call Opus and GPT-5 without limitations, if we removed the cost and latency of the calls from the equation. Most likely, the reliability is simply not there yet, especially for more niche scenarios (ones not directly related to coding or deep research).

But as we live in a world where you cannot just wave away cost and latency, my bet is on SLMs like `Nemotron 9B v2`, and surprisingly, SLMs and `BAML` are a match made in heaven. 

# Where to go from here

This notebook sets you up with all the functionality you might need to start experimenting with cutting-edge tools.

I myself am quite convinced as to [the value of smaller models](https://x.com/radekosmulski/status/1967386084868055175), which has been my motivation for this exploration.

I would highly recommend heading over to [the docs for BAML](https://docs.boundaryml.com/guide/introduction/what-is-baml) and checking out how you can modify the functionality I implemented in `baml_src/basic.baml`to better suite your needs.